<a href="https://colab.research.google.com/github/mralamdari/CV-Yolo/blob/main/YOLO_v3_ObjectDetection_TensorFlow_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import PIL
import struct # used to convert native Python data types such as strings and numbers into a string of bytes and vice versa
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.layers.merge import add, concatenate

In [2]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive'
!kaggle datasets download -d aruchomu/data-for-yolo-v3-kernel
!unzip \*.zip && rm *.zip

 96% 257M/267M [00:09<00:00, 24.8MB/s]
100% 267M/267M [00:09<00:00, 29.0MB/s]
Archive:  data-for-yolo-v3-kernel.zip
  inflating: coco.names              
  inflating: detections.gif          
  inflating: dog.jpg                 
  inflating: futur.ttf               
  inflating: office.jpg              
  inflating: yolov3.weights          


In [ ]:
class Weights_Reader:

  def __init__(self, filename):

    with open(filename, 'rb') as w_file:
      major, = struct.unpack('i', w_file.read(4))
      minor, = struct.unpack('i', w_file.read(4))
      revision, = struct.unpack('i', w_file.read(4))

      if (major*10 + minor) >= 2 and major < 1000 and minor < 1000:
        w_file.read(8)
      else:
        w_file.read(4)
      
      transpose = (major > 1000) or (minor > 1000)
      binary = w_file.read()
    
    self.offset = 0
    self.total_weights = np.frombuffer(binary, dtype='float32')

  def byte_reader(self, size):
    self.offset = self.offset + size
  
  def get_weights(self, model):

    for i in range(106): # 53*2 = 106 layers in total
      try: 
        convolutional_layer = model.load_layer(f'')